# CRUD Operations in SQLite

## Learning Objectives

By the end of this notebook, you will be able to:

1. Insert single and multiple rows with INSERT statements
2. Query data using SELECT with various options
3. Update existing records with UPDATE statements
4. Delete records with DELETE statements
5. Use parameterized queries to prevent SQL injection
6. Manage transactions with commit and rollback

## Setup: Create the Company Database

First, let's set up our company database with the three tables: departments, employees, and projects.

In [ ]:
import sqlite3
import os

# Remove existing database for fresh start
db_path = 'company.db'
if os.path.exists(db_path):
    os.remove(db_path)

# Create database and tables
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create tables
cursor.executescript('''
    CREATE TABLE departments (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL UNIQUE,
        budget REAL DEFAULT 0
    );
    
    CREATE TABLE employees (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        department_id INTEGER,
        salary REAL,
        hire_date TEXT,
        FOREIGN KEY (department_id) REFERENCES departments(id)
    );
    
    CREATE TABLE projects (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        department_id INTEGER,
        start_date TEXT,
        end_date TEXT,
        FOREIGN KEY (department_id) REFERENCES departments(id)
    );
''')

conn.commit()
print("Database and tables created successfully!")

## INSERT - Adding Data

The `INSERT` statement adds new rows to a table.

### Basic INSERT Syntax

```sql
INSERT INTO table_name (column1, column2, ...) VALUES (value1, value2, ...);
```

### Inserting a Single Row

In [ ]:
# Insert a single department
cursor.execute('''
    INSERT INTO departments (name, budget) VALUES ('Engineering', 500000)
''')

conn.commit()
print(f"Inserted department with ID: {cursor.lastrowid}")

In [ ]:
# Insert without specifying id (auto-generated)
cursor.execute('''
    INSERT INTO departments (name, budget) VALUES ('Marketing', 300000)
''')

cursor.execute('''
    INSERT INTO departments (name, budget) VALUES ('Sales', 400000)
''')

cursor.execute('''
    INSERT INTO departments (name, budget) VALUES ('HR', 200000)
''')

conn.commit()
print("Departments added!")

### Using Parameters (Parameterized Queries)

**ALWAYS** use parameters instead of string formatting to prevent SQL injection attacks.

In [ ]:
# GOOD: Using parameters with ?
employee_data = ('Alice Johnson', 1, 95000, '2020-03-15')
cursor.execute('''
    INSERT INTO employees (name, department_id, salary, hire_date)
    VALUES (?, ?, ?, ?)
''', employee_data)

conn.commit()
print(f"Inserted employee: {employee_data[0]}")

In [ ]:
# BAD: String formatting - VULNERABLE TO SQL INJECTION!
# NEVER do this:
# name = "Bob"
# cursor.execute(f"INSERT INTO employees (name) VALUES ('{name}')")

# What if name = "Bob'); DROP TABLE employees; --" ?
# This would delete your entire table!

print("Remember: Always use parameterized queries!")

### Named Parameters

You can also use named parameters with dictionaries, which makes code more readable.

In [ ]:
# Using named parameters with a dictionary
employee = {
    'name': 'Bob Smith',
    'dept_id': 1,
    'salary': 85000,
    'hire_date': '2021-06-01'
}

cursor.execute('''
    INSERT INTO employees (name, department_id, salary, hire_date)
    VALUES (:name, :dept_id, :salary, :hire_date)
''', employee)

conn.commit()
print(f"Inserted employee: {employee['name']}")

### Inserting Multiple Rows with executemany()

Use `executemany()` to efficiently insert multiple rows.

In [ ]:
# Insert multiple employees at once
employees = [
    ('Carol Williams', 1, 92000, '2019-08-20'),
    ('David Brown', 2, 78000, '2022-01-10'),
    ('Eva Martinez', 2, 82000, '2021-03-25'),
    ('Frank Wilson', 3, 88000, '2020-11-05'),
    ('Grace Lee', 3, 91000, '2019-05-12'),
    ('Henry Taylor', 4, 65000, '2022-07-18'),
    ('Ivy Chen', 1, 105000, '2018-02-28'),
    ('Jack Anderson', 3, 95000, '2020-09-14')
]

cursor.executemany('''
    INSERT INTO employees (name, department_id, salary, hire_date)
    VALUES (?, ?, ?, ?)
''', employees)

conn.commit()
print(f"Inserted {cursor.rowcount} employees")

In [ ]:
# Insert projects
projects = [
    ('Cloud Migration', 1, '2024-01-15', '2024-06-30'),
    ('Brand Refresh', 2, '2024-02-01', '2024-04-30'),
    ('Q2 Sales Campaign', 3, '2024-04-01', '2024-06-30'),
    ('Mobile App v2.0', 1, '2024-03-01', '2024-09-30'),
    ('Employee Onboarding System', 4, '2024-02-15', '2024-05-31'),
    ('Data Analytics Platform', 1, '2024-05-01', '2024-12-31')
]

cursor.executemany('''
    INSERT INTO projects (name, department_id, start_date, end_date)
    VALUES (?, ?, ?, ?)
''', projects)

conn.commit()
print(f"Inserted {cursor.rowcount} projects")

## SELECT - Reading Data

The `SELECT` statement retrieves data from tables.

### Basic SELECT Syntax

```sql
SELECT column1, column2, ... FROM table_name;
SELECT * FROM table_name;  -- All columns
```

### Selecting All Columns

In [ ]:
# Select all departments
cursor.execute('SELECT * FROM departments')

# fetchall() returns all results as a list of tuples
departments = cursor.fetchall()

print("All Departments:")
print(f"{'ID':<4} {'Name':<15} {'Budget':>10}")
print("-" * 32)
for dept in departments:
    print(f"{dept[0]:<4} {dept[1]:<15} ${dept[2]:>9,.0f}")

### Selecting Specific Columns

In [ ]:
# Select only name and salary columns
cursor.execute('SELECT name, salary FROM employees')

print("Employee Names and Salaries:")
print(f"{'Name':<20} {'Salary':>10}")
print("-" * 32)
for row in cursor.fetchall():
    print(f"{row[0]:<20} ${row[1]:>9,.0f}")

### Fetching Methods

- `fetchone()` - Returns the next row or None
- `fetchall()` - Returns all remaining rows as a list
- `fetchmany(n)` - Returns the next n rows

In [ ]:
# fetchone() - get one row at a time
cursor.execute('SELECT name FROM employees')

print("First three employees (using fetchone):")
print(cursor.fetchone()[0])
print(cursor.fetchone()[0])
print(cursor.fetchone()[0])

In [ ]:
# fetchmany() - get n rows at a time
cursor.execute('SELECT name FROM employees')

print("\nFirst batch of 3:")
batch1 = cursor.fetchmany(3)
for row in batch1:
    print(f"  {row[0]}")

print("\nSecond batch of 3:")
batch2 = cursor.fetchmany(3)
for row in batch2:
    print(f"  {row[0]}")

### Iterating Over Results

Cursors are iterable - you can loop directly over them.

In [ ]:
# Iterate directly over cursor (memory efficient for large results)
cursor.execute('SELECT name, salary FROM employees')

print("Employees earning over $90,000:")
for name, salary in cursor:
    if salary > 90000:
        print(f"  {name}: ${salary:,.0f}")

### Row Factory: Getting Results as Dictionaries

By default, results are tuples. You can change this with `row_factory`.

In [ ]:
# Use Row factory for named access
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

cursor.execute('SELECT * FROM employees WHERE id = 1')
employee = cursor.fetchone()

# Access by column name
print(f"Name: {employee['name']}")
print(f"Salary: ${employee['salary']:,.0f}")
print(f"Hire Date: {employee['hire_date']}")

# Get column names
print(f"\nColumns: {employee.keys()}")

In [ ]:
# Reset row_factory for the rest of the notebook
conn.row_factory = None
cursor = conn.cursor()

## UPDATE - Modifying Data

The `UPDATE` statement modifies existing rows.

### Basic UPDATE Syntax

```sql
UPDATE table_name SET column1 = value1, column2 = value2, ... WHERE condition;
```

**WARNING**: Always use a WHERE clause! Without it, ALL rows will be updated.

In [ ]:
# Update a single employee's salary
cursor.execute('SELECT name, salary FROM employees WHERE id = 1')
before = cursor.fetchone()
print(f"Before: {before[0]} - ${before[1]:,.0f}")

# Give Alice a raise
cursor.execute('''
    UPDATE employees SET salary = 100000 WHERE id = 1
''')
conn.commit()

cursor.execute('SELECT name, salary FROM employees WHERE id = 1')
after = cursor.fetchone()
print(f"After: {after[0]} - ${after[1]:,.0f}")
print(f"Rows affected: {cursor.rowcount}")

In [ ]:
# Update multiple columns at once
cursor.execute('''
    UPDATE departments 
    SET budget = 550000 
    WHERE name = 'Engineering'
''')
conn.commit()

cursor.execute('SELECT * FROM departments WHERE name = "Engineering"')
print(cursor.fetchone())

In [ ]:
# Update multiple rows matching a condition
# Give everyone in Engineering a 5% raise
cursor.execute('''
    UPDATE employees 
    SET salary = salary * 1.05 
    WHERE department_id = 1
''')
conn.commit()
print(f"Engineering employees updated: {cursor.rowcount}")

# Verify
cursor.execute('SELECT name, salary FROM employees WHERE department_id = 1')
print("\nEngineering salaries after raise:")
for name, salary in cursor.fetchall():
    print(f"  {name}: ${salary:,.0f}")

### Update with Parameters

In [ ]:
# Using parameters in UPDATE
new_budget = 350000
dept_name = 'Marketing'

cursor.execute('''
    UPDATE departments SET budget = ? WHERE name = ?
''', (new_budget, dept_name))

conn.commit()
print(f"Updated {dept_name} budget to ${new_budget:,}")

## DELETE - Removing Data

The `DELETE` statement removes rows from a table.

### Basic DELETE Syntax

```sql
DELETE FROM table_name WHERE condition;
```

**WARNING**: Always use a WHERE clause! Without it, ALL rows will be deleted.

In [ ]:
# First, add a temporary employee to delete
cursor.execute('''
    INSERT INTO employees (name, department_id, salary, hire_date)
    VALUES ('Temp Worker', 1, 50000, '2024-01-01')
''')
temp_id = cursor.lastrowid
conn.commit()

# Count employees
cursor.execute('SELECT COUNT(*) FROM employees')
print(f"Employees before delete: {cursor.fetchone()[0]}")

# Delete the temporary employee
cursor.execute('DELETE FROM employees WHERE id = ?', (temp_id,))
conn.commit()

cursor.execute('SELECT COUNT(*) FROM employees')
print(f"Employees after delete: {cursor.fetchone()[0]}")
print(f"Rows deleted: {cursor.rowcount}")

In [ ]:
# Delete with a condition
# First, add some test data
cursor.executemany('''
    INSERT INTO employees (name, department_id, salary, hire_date)
    VALUES (?, ?, ?, ?)
''', [
    ('Intern 1', 1, 30000, '2024-06-01'),
    ('Intern 2', 2, 30000, '2024-06-01'),
    ('Intern 3', 3, 30000, '2024-06-01')
])
conn.commit()

# Delete all employees with salary under 40000
cursor.execute('DELETE FROM employees WHERE salary < 40000')
conn.commit()
print(f"Deleted {cursor.rowcount} low-salary employees")

## Transactions: Commit and Rollback

A **transaction** groups multiple SQL operations into a single unit. Either all operations succeed, or none of them do.

- `commit()` - Save all changes since the last commit
- `rollback()` - Undo all changes since the last commit

In [ ]:
# Demonstrate rollback
cursor.execute('SELECT COUNT(*) FROM employees')
count_before = cursor.fetchone()[0]
print(f"Employees before: {count_before}")

# Insert some data (not committed)
cursor.execute("INSERT INTO employees (name, salary) VALUES ('Test1', 50000)")
cursor.execute("INSERT INTO employees (name, salary) VALUES ('Test2', 50000)")

cursor.execute('SELECT COUNT(*) FROM employees')
count_during = cursor.fetchone()[0]
print(f"Employees during (uncommitted): {count_during}")

# Rollback - undo the inserts
conn.rollback()

cursor.execute('SELECT COUNT(*) FROM employees')
count_after = cursor.fetchone()[0]
print(f"Employees after rollback: {count_after}")

In [ ]:
# Transaction example: Transfer budget between departments
# This should be atomic - both updates succeed or neither does

def transfer_budget(from_dept: str, to_dept: str, amount: float) -> bool:
    """Transfer budget between departments atomically."""
    try:
        # Reduce budget from source department
        cursor.execute('''
            UPDATE departments SET budget = budget - ? WHERE name = ?
        ''', (amount, from_dept))
        
        if cursor.rowcount == 0:
            raise ValueError(f"Department '{from_dept}' not found")
        
        # Add budget to destination department
        cursor.execute('''
            UPDATE departments SET budget = budget + ? WHERE name = ?
        ''', (amount, to_dept))
        
        if cursor.rowcount == 0:
            raise ValueError(f"Department '{to_dept}' not found")
        
        # Both succeeded - commit
        conn.commit()
        print(f"Transferred ${amount:,.0f} from {from_dept} to {to_dept}")
        return True
        
    except Exception as e:
        # Something went wrong - rollback
        conn.rollback()
        print(f"Transfer failed: {e}")
        return False

# Show budgets before
cursor.execute('SELECT name, budget FROM departments ORDER BY name')
print("Budgets before:")
for name, budget in cursor.fetchall():
    print(f"  {name}: ${budget:,.0f}")

# Perform transfer
print()
transfer_budget('Engineering', 'Marketing', 50000)

# Show budgets after
print("\nBudgets after:")
cursor.execute('SELECT name, budget FROM departments ORDER BY name')
for name, budget in cursor.fetchall():
    print(f"  {name}: ${budget:,.0f}")

In [ ]:
# Try a transfer to non-existent department (should rollback)
print("Attempting transfer to non-existent department:")
transfer_budget('Marketing', 'Finance', 10000)

# Marketing budget should be unchanged
cursor.execute('SELECT budget FROM departments WHERE name = "Marketing"')
print(f"Marketing budget (unchanged): ${cursor.fetchone()[0]:,.0f}")

## SQL Injection Prevention

SQL injection is a common security vulnerability. **Always** use parameterized queries.

In [ ]:
# Example of what SQL injection looks like
def unsafe_search(name: str) -> list:
    """UNSAFE: Don't do this! Vulnerable to SQL injection."""
    # BAD - using string formatting
    query = f"SELECT * FROM employees WHERE name = '{name}'"
    print(f"Unsafe query: {query}")
    # cursor.execute(query)  # Don't run this!
    return []

def safe_search(name: str) -> list:
    """SAFE: Uses parameterized query."""
    cursor.execute("SELECT * FROM employees WHERE name = ?", (name,))
    return cursor.fetchall()

# Normal input
print("Normal search:")
unsafe_search("Alice Johnson")

# Malicious input
print("\nMalicious input (would delete all data!):")
malicious = "'; DROP TABLE employees; --"
unsafe_search(malicious)

# Safe search handles it correctly
print("\nSafe search with same input:")
result = safe_search(malicious)  # Returns empty - no match, but no damage!
print(f"Results: {result}")

## Summary

In this notebook, you learned:

### CRUD Operations

| Operation | SQL | Python Method |
|-----------|-----|---------------|
| **C**reate | `INSERT INTO ... VALUES ...` | `execute()`, `executemany()` |
| **R**ead | `SELECT ... FROM ...` | `fetchone()`, `fetchall()`, `fetchmany()` |
| **U**pdate | `UPDATE ... SET ... WHERE ...` | `execute()` |
| **D**elete | `DELETE FROM ... WHERE ...` | `execute()` |

### Key Concepts

1. **Parameters** - Use `?` or `:name` placeholders to prevent SQL injection
2. **executemany()** - Efficient batch inserts
3. **Transactions** - `commit()` saves changes, `rollback()` undoes them
4. **Row factory** - Get results as dictionaries with `sqlite3.Row`
5. **Always use WHERE** - Prevent accidental updates/deletes of all rows

## Exercises

### Exercise 1: Insert New Data

Insert a new department called "Research" with a budget of $600,000.

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
cursor.execute('''
    INSERT INTO departments (name, budget) VALUES (?, ?)
''', ('Research', 600000))
conn.commit()

# Verify
cursor.execute('SELECT * FROM departments WHERE name = "Research"')
print(cursor.fetchone())
```
</details>

### Exercise 2: Batch Insert

Insert three new employees using `executemany()`. Each should have a name, department_id, salary, and hire_date.

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
new_employees = [
    ('Karen White', 2, 75000, '2024-02-01'),
    ('Leo Garcia', 3, 82000, '2024-01-15'),
    ('Mia Davis', 4, 68000, '2024-03-01')
]

cursor.executemany('''
    INSERT INTO employees (name, department_id, salary, hire_date)
    VALUES (?, ?, ?, ?)
''', new_employees)
conn.commit()

print(f"Inserted {cursor.rowcount} employees")

# Verify
cursor.execute('SELECT name FROM employees ORDER BY id DESC LIMIT 3')
for row in cursor.fetchall():
    print(f"  - {row[0]}")
```
</details>

### Exercise 3: Update with Condition

Give all employees in the Sales department (department_id = 3) a 10% raise.

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
# Show before
cursor.execute('SELECT name, salary FROM employees WHERE department_id = 3')
print("Before raise:")
for name, salary in cursor.fetchall():
    print(f"  {name}: ${salary:,.0f}")

# Update
cursor.execute('''
    UPDATE employees SET salary = salary * 1.10 WHERE department_id = 3
''')
conn.commit()
print(f"\nUpdated {cursor.rowcount} employees")

# Show after
cursor.execute('SELECT name, salary FROM employees WHERE department_id = 3')
print("\nAfter raise:")
for name, salary in cursor.fetchall():
    print(f"  {name}: ${salary:,.0f}")
```
</details>

### Exercise 4: Safe Delete Function

Write a function that deletes an employee by name, using parameterized queries. Return the number of rows deleted.

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
def delete_employee_by_name(name: str) -> int:
    """Safely delete an employee by name using parameterized query."""
    cursor.execute('DELETE FROM employees WHERE name = ?', (name,))
    conn.commit()
    return cursor.rowcount

# Add a test employee
cursor.execute("INSERT INTO employees (name, salary) VALUES ('Test Delete', 40000)")
conn.commit()

# Delete them
deleted = delete_employee_by_name('Test Delete')
print(f"Deleted {deleted} employee(s)")

# Try to delete non-existent
deleted = delete_employee_by_name('Does Not Exist')
print(f"Deleted {deleted} employee(s)")
```
</details>

### Exercise 5: Transaction Practice

Write a function that transfers salary budget from one employee to another. If either employee doesn't exist, roll back the transaction.

In [ ]:
# Your code here


<details>
<summary>Click to see solution</summary>

```python
def transfer_salary(from_id: int, to_id: int, amount: float) -> bool:
    """Transfer salary between employees atomically."""
    try:
        # Reduce salary from source
        cursor.execute(
            'UPDATE employees SET salary = salary - ? WHERE id = ?',
            (amount, from_id)
        )
        if cursor.rowcount == 0:
            raise ValueError(f"Employee {from_id} not found")
        
        # Add salary to destination
        cursor.execute(
            'UPDATE employees SET salary = salary + ? WHERE id = ?',
            (amount, to_id)
        )
        if cursor.rowcount == 0:
            raise ValueError(f"Employee {to_id} not found")
        
        conn.commit()
        print(f"Transferred ${amount:,.0f} from employee {from_id} to {to_id}")
        return True
        
    except Exception as e:
        conn.rollback()
        print(f"Transfer failed: {e}")
        return False

# Test successful transfer
cursor.execute('SELECT id, name, salary FROM employees LIMIT 2')
emp1, emp2 = cursor.fetchall()
print(f"Before: {emp1[1]}: ${emp1[2]:,.0f}, {emp2[1]}: ${emp2[2]:,.0f}")

transfer_salary(emp1[0], emp2[0], 5000)

cursor.execute('SELECT name, salary FROM employees WHERE id IN (?, ?)', (emp1[0], emp2[0]))
for name, salary in cursor.fetchall():
    print(f"After: {name}: ${salary:,.0f}")

# Test failed transfer
print("\nTrying invalid transfer:")
transfer_salary(emp1[0], 9999, 1000)  # Non-existent employee
```
</details>

## Next Steps

In the next notebook, **03_queries_and_joins.ipynb**, you'll learn:
- Filtering data with WHERE clauses
- Sorting and limiting results
- Aggregate functions (COUNT, SUM, AVG)
- Grouping data with GROUP BY
- Joining multiple tables together

## Cleanup

In [ ]:
# Close connection and remove database
conn.close()

if os.path.exists('company.db'):
    os.remove('company.db')
    print("Database file removed.")